# Исследование надежности заемщиков



Проведем исследование, влияет ли семейное положение и количество детей заемщиков на факт возврата кредита в срок, на основе статистики о платёжеспособности клиентов банка.


## Откроем таблицу и изучим общую информацию о данных

1. Импортируем библиотеку pandas. Считайте данные из csv-файла в датафрейм и сохраним в переменную `data`. Путь к файлу:**

`/datasets/data.csv`

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

2. Выведем первые 20 строчек датафрейма `data` на экран.**

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


3. Выведем основную информацию о датафрейме с помощью метода `info()`.**

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

4. Выведем количество пропущенных значений для каждого столбца. 

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

5. В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце мы обработаем на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

6. В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода `abs()`.**

In [6]:
data['days_employed'] = data['days_employed'].abs()

7. Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.**

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

8. Выведем перечень уникальных значений столбца `children`.**

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

9. В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

10. Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

11. Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

12. Убедимся, что все пропуски заполнены. Проверим себя и ещё раз выведем количество пропущенных значений для каждого столбца с помощью двух методов.**

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

13. Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

14. Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру. Проверим остальные столбцы.**

In [14]:
data['education'] = data['education'].str.lower()

15. Выведем на экран количество строк-дубликатов в данных, удалим их.**

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

16. На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.




In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

17. Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

18. Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.



In [2]:
def categorize_purpose(row):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [13]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

data.head(150)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,B,None
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,C,None
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,C,None
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,B,None
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,C,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,0,NaN,62,среднее,1,женат / замужем,0,M,пенсионер,0,NaN,строительство недвижимости,None,None
146,0,-1302.026042,46,среднее,1,женат / замужем,0,F,сотрудник,0,98709.730547,покупка жилья,C,None
147,0,-5772.479896,47,среднее,1,женат / замужем,0,F,госслужащий,1,74535.097331,операции с недвижимостью,C,None
148,0,-5050.292840,37,среднее,1,женат / замужем,0,F,сотрудник,0,145159.214977,на покупку своего автомобиля,C,None


### Шаг 3. Исследуйте данные и ответьте на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [22]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')
def debt_sort(row):             #создала функцию, чтобы уменьшить количество категорий, исходя из количества детей
        if row == 0:
            return "бездетные"
        if row == 1:
            return '1 ребенок'
        if row == 2:
            return '2 ребенка'
        if row >= 3:
            return 'многодетные'
        return '-' 
data['chidren_sort'] = data['children'].apply(debt_sort)
print(data.groupby('chidren_sort').agg(debt_count=('debt','count'), debt_mean=('debt','mean')))  #вывела на экран таблицу, где видно количество кредитов и долю невозврата


              debt_count  debt_mean
chidren_sort                       
-                     47   0.021277
1 ребенок           4818   0.092154
2 ребенка           2055   0.094404
бездетные          14149   0.075129
многодетные          456   0.085526


**Вывод:** зависимость между количеством детей и возвратом кредита в срок просматривается, но не очень значительная. 
Наименьший процент задержек у бездетных: 7,5%;
несколько больший процент задержек у многодетных:8,5%;
наибольший процент задержек у семей, имеющих 1 и 2 ребенка: 9,2 и 9,4% соответственно

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [23]:
print(data.groupby('family_status').agg(debt_count=('debt','count'), debt_mean=('debt','mean'))) #вывела на экран таблицу, где видно количество кредитов и долю невозврата по разным семейным статусам

                       debt_count  debt_mean
family_status                               
Не женат / не замужем        2813   0.097405
в разводе                    1195   0.071130
вдовец / вдова                960   0.065625
гражданский брак             4177   0.092890
женат / замужем             12380   0.075202


**Вывод:** зависимость между семейным положением и возвратом кредита в срок просматривается. Те, кто не состоят и не состояли в официально серьезных отношениях чаще имели задолженности по кредиту (более, чем на 2%)

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [8]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass
data['total_income_category'] = data['total_income'].apply(categorize_income)
print(data.groupby('total_income_category').agg(debt_count=('debt','count'), debt_mean=('debt','mean'))) #вывела на экран таблицу, где видно количество кредитов и долю невозврата по разным уровням дохода

                       debt_count  debt_mean
total_income_category                       
A                              25   0.080000
B                            5041   0.070621
C                           13913   0.085532
D                             350   0.060000
E                              22   0.090909


**Вывод:** лучше всего возвращают долги граждане с достатком от 30 001 руб. до 50 000 руб.- 6% задерживают возврат кредитов.
Хуже всего возвращают долги граждане с зарплатой менее или равной 30 000 руб. - 9% задержек.
Граждане с зарплатой 50 001-200 000 руб. имеют задержки в 8,6%, с зарплатой 200 001-1000 000 - 7,1%, с зарплатой более 1 000 000 руб. - 8%.

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [25]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'автомобиль'
        elif 'жил' in row or 'недвиж' in row: 
            return 'недвижимость'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'
data['purpose_category'] = data['purpose'].apply(categorize_purpose)    
print(data.groupby('purpose_category').agg(debt_count=('debt','count'), debt_mean=('debt','mean'))) #вывела на экран таблицу, где видно количество кредитов и долю невозврата в зависимости от цели кредита

                       debt_count  debt_mean
purpose_category                            
автомобиль                   4315   0.093395
недвижимость                10840   0.072140
получение образования        4022   0.091994
проведение свадьбы           2348   0.079216


**Вывод:** при взятии кредита на недвижиость меньше всего просрочек (7,2%), на автомобиль - больше всего (9,3%). 
Если цель кредита - образование, то задержки с возвратом кредита составляют 9,2%, если проведение свадьбы - 7,9%.

#### 3.5 Приведем возможные причины появления пропусков в исходных данных.

*Ответ:* случайная ошибка человека, пользователи/компании специально не хотят указывать какие-то свои данные и т.д.

  В нашем конкретном случае пропуски идут в столбцах 'days employed' и 'total income', соответственно наиболее вероятная причина пропусков: отсутствие трудового стажа и дохода, то есть реально это '0'. Такие пропуски попадаются у пенсионеров. Предполагаю, что расчеты подразумевали расчет стажа за последние несколько лет, например, а пенсионеры уже не работали в этот период. 
Но, даже если данные пенсонеры вообще не работали, в реальности пенсию им, все равно, платят в России, поэтому емемесячный доход у них, все равно, есть. Предполагаю, что либо банк, либо пенсионеры не рассматривают социальные выплаты, как доход, поэтому не указали данные.
(Правда, из реального опыта, знаю, что банки считают пенсию доходом).

### Шаг 4: общий вывод.

Было проведено исследование с целью определения влияния различных факторов на своевременность возврата кредита.
Результаты данного исследования могут быть использованы банками для построения модели кредитного скрининга.
 
 ### В ходе исследования проверялись следующие гипотезы: #
 1. Есть зависимость между количеством детей и возвратом кредита в срок;
 2. Есть зависимость между семейным положением и возвратом кредита в срок;
 3. Есть зависимость между уровнем дохода и возвратом кредита в срок;
 4. Есть завтсимость между целью кредита и возвратом кредита в срок.
 
Гипотеза 1 частично подтверждена:*мало данных по многодетным.
Наименьший процент задержек у бездетных: 7,5%; 
несколько больший процент задержек у многодетных: 8,5%; 
наибольший процент задержек у семей, имеющих 1 и 2 ребенка: 9,2 и 9,4% соответственно.

Рекомендуется изучить больше данных по гражданам с 3 и более детьми.

Гипотеза 2 подтверждена.
Граждане, которые не состоят и не состояли в официально серьезных отношениях, чаще имели задолженности по кредиту: на 2% процента больше просрочек по кредитам, чем у граждан, которые состоях или состояли ранее в официальном браке.

Гипотеза 3 частично подтверждена: *мало данных о граждан с доходом выше 1000 000 руб., а также с доходом менее 30 000 руб.
Лучше всего возвращают долги граждане с достатком от 30 001 руб. до 50 000 руб.- 6% задерживают возврат кредитов;
хуже всего возвращают долги граждане с зарплатой менее или равной 30 000 руб. - 9% задержек;
граждане с зарплатой 50 001-200 000 руб. имеют задержки в 8,6%;
с зарплатой 200 001-1000 000 - 7,1%;
с зарплатой более 1 000 000 руб. - 8%.

Рекомендуется изучить больше данных по гражданам с зарплатой выше 1000 000 руб., а также с зарплатой менее 30 000 руб.

Гипотеза 4 подтверждена.
При взятии кредита на недвижиость меньше всего просрочек (7,2%);
на автомобиль - больше всего (9,3%); 
когда цель кредита - образование, то задержки с возвратом кредита составляют 9,2%, 
когда цель кредита - проведение свадьбы - 7,9%.

Общие рекомендации:

1. Специалистам, которые занимаются построением модели кредитного скрининга, можно взять за основу выводы по 2 и 4 гипотезе.
2. Рекомендуется собрать больше информации для повторной проверки 1 и 3 гипотезы.

